In [1]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: phy. Use `wandb login --relogin` to force relogin


True

In [2]:
import yaml 
cfg = yaml.safe_load(open('sample.yaml'))
cfg

{'saving_dir': 'experiments_1',
 'dataset': 'MNIST',
 'split': 'iid',
 'num_clients': 10,
 'imbalanced_ratio': 5,
 'corruption_prob': 0.5,
 'algorithm': 'FedAvg',
 'model': 'ConvNet2',
 'use_gpu': True,
 'epoch': 100,
 'batch_size': 32,
 'test_batch_size': 1024,
 'lr': 0.01,
 'local_epoch': 5,
 'num_clients_per_round': 5,
 'log_freq': 100}

In [3]:
from Model import ConvNet2

model = ConvNet2(in_channels=3, h=28, w=28, hidden=2048, class_num=10,  dropout=.0)
dataset_name = cfg['dataset']
from Data import load_centralized_dataset
trainset, testset = load_centralized_dataset(dataset_name, validation_split=0, download=False)


In [4]:
import torchvision
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

In [5]:
from Split import IIDSplitter
client_num = 10
iid_splliter = IIDSplitter(client_num)
clients_dataidx_map = iid_splliter(trainset)

Splitting dataset into 10 clients.
{0: 6000, 1: 6000, 2: 6000, 3: 6000, 4: 6000, 5: 6000, 6: 6000, 7: 6000, 8: 6000, 9: 6000}


In [6]:
from Trainer import LocalTrainer
with wandb.init(project='debug', config=cfg):
    trainer = LocalTrainer(model, clients_dataidx_map, trainset, testset, transforms, cfg)                 
    trainer.train()
    trainer.evaluate()


KeyError: 'local_round'